In [ ]:
! pip install cohere -q
import cohere
co = cohere.ClientV2("") # Get your free API key: https://dashboard.cohere.com/api-keys

In [ ]:
# Mock databases for available meeting slots
import json
import os

meeting_slots_database = {
    "2024-10-15": [
        {"start": "14:00", "end": "15:00"},
        {"start": "15:30", "end": "16:30"},
        {"start": "17:00", "end": "18:00"}
    ]
}

# Function definitions to interact with the tools
def query_meeting_slots(day: str) -> list:
    """
    Function to retrieve available meeting slots for the given day
    """
    slots = meeting_slots_database.get(day, [])
    if slots:
        return [{
            "date": day,
            "available_slots": slots
        }]
    else:
        return [{"date": day, "summary": "No available slots for this day."}]

def create_meeting(day: str, time: str, duration: int) -> dict:
    """
    Function to schedule a meeting for the given time and duration
    """
    return {
        "date": day,
        "time": time,
        "duration": duration,
        "confirmation": f"Meeting scheduled for {duration} hour(s) starting at {time} on {day}."
    }

# Map tools to their respective functions
functions_map = {
    "query_meeting_slots": query_meeting_slots,
    "create_meeting": create_meeting,
}

# Tool definitions based on provided structure
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_meeting_slots",
            "description": "Retrieve available meeting slots for a specified day.",
            "parameters": {
                "type": "object",
                "properties": {
                    "day": {
                        "type": "string",
                        "description": "Retrieve meeting slots for this day, formatted as YYYY-MM-DD."
                    }
                },
                "required": ["day"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_meeting",
            "description": "Schedule a meeting at the specified time on the given day.",
            "parameters": {
                "type": "object",
                "properties": {
                    "day": {
                        "type": "string",
                        "description": "The date of the meeting, formatted as YYYY-MM-DD."
                    },
                    "time": {
                        "type": "string",
                        "description": "The start time of the meeting, formatted in 24-hour military time."
                    },
                    "duration": {
                        "type": "integer",
                        "description": "The duration of the meeting in hours."
                    }
                },
                "required": ["day", "time", "duration"]
            }
        }
    }
]

# Conversation context
system_message = """
## Task & Context
You assist employees at DragonForge Games by helping them schedule meetings with the game design team. You provide meeting slots and book meetings on request.
"""

# Initial user request
user_message = "Can you help me schedule a meeting with the game design team for today after 3 PM?"

# Add system and user messages
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

# Step 1: Model generates the tool plan for querying meeting slots
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages,
                   tools=tools)

print("Tool plan generated by the model:")
print(response.message.tool_plan)

# Append the tool calls and plan to the chat history
messages.append(
    {
        "role": "assistant",
        "tool_calls": response.message.tool_calls,
        "tool_plan": response.message.tool_plan
    }
)

# Step 2: Execute the tool calls recommended by the model
for tc in response.message.tool_calls:
    tool_result = functions_map[tc.function.name](**json.loads(tc.function.arguments))

    # Store tool result to send back to the model
    tool_content = []
    for data in tool_result:
        tool_content.append({"type": "document", "document": {"data": json.dumps(data)}})

    # Append tool result to the chat history
    messages.append(
        {"role": "tool", "tool_call_id": tc.id, "content": tool_content}
    )

    print("Tool result to be fed back to the model:")
    for result in tool_content:
        print(result)

# Step 3: Send the tool results back to the model
response = co.chat(
    model="command-r-plus-08-2024",
    messages=messages,
    tools=tools
)

# Final model response based on the tool results
print("Final answer:")
print(response.message.content[0].text)